In [1]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import json

from config import *

uri = URI

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['datashop']
db.list_collection_names?

Signature:
db.list_collection_names(
    session: 'Optional[ClientSession]' = None,
    filter: 'Optional[Mapping[str, Any]]' = None,
    comment: 'Optional[Any]' = None,
    **kwargs: 'Any',
) -> 'list[str]'
Docstring:
Get a list of all the collection names in this database.

For example, to list all non-system collections::

    filter = {"name": {"$regex": r"^(?!system\.)"}}
    db.list_collection_names(filter=filter)

:param session: a
    :class:`~pymongo.client_session.ClientSession`.
:param filter:  A query document to filter the list of
    collections returned from the listCollections command.
:param comment: A user-provided comment to attach to this
    command.
:param kwargs: Optional parameters of the
    `listCollections command
    <https://mongodb.com/docs/manual/reference/command/listCollections/>`_
    can be passed as keyword arguments to this method. The supported
    options differ by server version.


.. versionchanged:: 3.8
   Added the ``filter`` and ``**kwargs``

In [ ]:
!pip install -q fastapi[all] uvicorn

In [ ]:
result = shopee.aggregate(
    [
    {
        "$unwind": "$responseBody.data.item"  # Dùng unwind để tách các item trong list
    },
    {
        "$group": {
            "_id": "$responseBody.data.item.item_id",  # Nhóm theo item_id
            "prices": {"$push": "$responseBody.data.item.price"},  # Lấy danh sách giá sản phẩm
            "scraped_times": {"$push": "$responseBody.data.item.scraped_timestamp"}  # Lấy danh sách thời gian scraped
        }
    },
    {
        "$project": {
            "_id": 1,  # item_id
            "prices": 1,  # Giá của sản phẩm
            "scraped_times": 1,  # Thời gian scraped
            "distinct_prices": {"$size": "$prices"}  # Tính số lượng giá khác nhau cho mỗi sản phẩm
        }
    }
]
)

In [3]:
# Truy vấn MongoDB để lấy giá sản phẩm và thời gian scraped theo ngày
pipeline = [
    {
        "$unwind": "$responseBody.data.item"  # Dùng unwind để tách các item trong list
    },
    {
        "$project": {
            "_id": "$responseBody.data.item.item_id",  # item_id
            "price": "$responseBody.data.item.price",  # Giá của sản phẩm
            "scraped_timestamp": "$scraped_timestamp",  # Thời gian scraped ở ngoài cấp item
            # Chuyển scraped_timestamp thành ngày (nếu là string)
            "scraped_date": {
                "$dateToString": {"format": "%Y-%m-%d", "date": {"$toDate": "$scraped_timestamp"}}
            }
        }
    },
    {
        "$group": {
            "_id": {"item_id": "$_id", "scraped_date": "$scraped_date"},  # Nhóm theo item_id và ngày scraped
            "prices": {"$push": "$price"},  # Lấy danh sách giá sản phẩm
            "distinct_prices": {"$addToSet": "$price"}  # Lấy các giá distinct
        }
    },
    {
        "$project": {
            "_id": 0,
            "item_id": "$_id.item_id",  # item_id
            "scraped_date": "$_id.scraped_date",  # scraped_date
            "prices": 1,  # Giá sản phẩm
            "distinct_prices_count": {"$size": "$distinct_prices"}  # Số lượng giá distinct trong ngày
        }
    },
    {
        "$sort": {
            "item_id": 1,  # Sắp xếp theo item_id
            "scraped_date": 1  # Sắp xếp theo ngày scraped
        }
    }
]

# Thực thi pipeline
result = list(shopee.aggregate(pipeline))
import pandas as pd
df = pd.DataFrame(result)
del pd # remove namespaces
df['item_id'].unique()

array([ 5873954476, 24509035804, 27253416536, 27312277956, 27603632331,
       29560903606])

In [2]:
shopee = db['shopee']
result = shopee.find_one({})

In [4]:
df

,prices,item_id,scraped_date,distinct_prices_count
0,[1889000000000],5873954476,2024-11-14,1
1,[1859000000000],5873954476,2024-11-15,1
2,[1889000000000],5873954476,2024-11-16,1
3,[1889000000000],5873954476,2024-11-17,1
4,[1889000000000],5873954476,2024-11-18,1
...,...,...,...,...
97,[3499900000000],29560903606,2024-11-26,1
98,[3499900000000],29560903606,2024-11-27,1
99,[3499900000000],29560903606,2024-11-28,1
100,[3489000000000],29560903606,2024-11-29,1


In [15]:
shopee.distinct('responseBody.data.item.item_id')

[5873954476, 24509035804, 27253416536, 27312277956, 27603632331, 29560903606]

In [13]:
tiki = db['tiki']
result = tiki.find_one()
len(tiki.distinct('id'))

14

In [14]:
lazada = db['lazada']
lazada.distinct('responseBody.itemId')
lazada.distinct('responseBody.reviews')

[1040858590, 2730432618, 2792189799]

In [66]:
# // Assuming structure:
# // {
# //   responseBody: {
# //     itemId: string,
# //     reviews: array
# //   }
# // }

review_on_lazada = db.lazada.aggregate([
  # Unwind the reviews array to create separate documents
  {
    "$unwind": "$responseBody.reviews"
  },
  
  # Group by itemId
  {
    "$group": {
      "_id": "$responseBody.itemId",
      "reviews": { 
        "$push": "$responseBody.reviews"
      },
      "totalReviews": { 
        "$sum": 1 
      },
      # If ratingCountByScore is an object, we should sum its values
      "ratingCountByScore": { 
        "$first": "$responseBody.ratingCountByScore"
      }    }
  },
  
  # Optional: Sort by number of reviews (descending)
  {
    "$sort": {
      "totalReviews": -1
    }
  }
])

In [64]:
list(review_on_lazada)[0].keys()

dict_keys(['_id', 'reviews', 'totalReviews', 'ratingCountByScore'])

In [67]:
from pprint import pprint

# Để xem cấu trúc JSON 
pprint(list(review_on_lazada), indent=2, width=95)

[ { '_id': 1040858590,
    'ratingCountByScore': {'1': 28, '2': 5, '3': 10, '4': 17, '5': 627},
    'reviews': [ { 'images': [ { 'isApproved': True,
                                 'isLegal': True,
                                 'url': 'https://lzd-u.slatic.net/64d0f2ae761b447cbd9a8225c107316e_3_1719075438.277826.jpg'}],
                   'isHelpful': True,
                   'isPurchased': True,
                   'itemId': 1040858590,
                   'likeCount': 0,
                   'rating': 5,
                   'reviewContent': 'Đã nhận hàng trong tình trạng full team seal, đóng gói '
                                    'rất cẩn thận! Giao hàng khá nhanh, tối chốt mà sang hôm '
                                    'sau hàng đã về tới Thảo Điền, quá nhanh quá nguy hiểm '
                                    '😂! \n'
                                    'Chưa dùng, về chất lượng sẽ cho quánh giá sau!',
                   'reviewRateId': 424016786958590,
                   'revi

In [77]:
# skus của 3 sản phẩm từ lazada
lazada.aggregate(
  [
    {
      "$project": {
        "itemId": "$responseBody.itemId",
        "skus": "$responseBody.skus"
      }
    }
  ]
)

In [85]:
# truy vấn sku's specifications của 3 product
desc_by_id = lazada.aggregate(
  [
        # Group by itemId
        {
            "$group": {
              "_id": "$responseBody.itemId",
              "desc": {
                "$push": {
                  "$ifNull": [
                    {"$arrayElemAt": ["$responseBody.skus.specifications", 0]},  # Safely get the second sku's specifications
                    {}  # Default to empty object if specifications not found
                  ]
                }
              }
            }
        }
  ]
)

In [87]:
pipeline = [
    {
        "$unwind": "$responseBody.skus"  # Dùng unwind để tách các phần tử trong mảng skus
    },
    {
        "$project": {
            "_id": "$responseBody.itemId",  # itemId là trường bạn dùng để nhóm
            "salePrice": "$responseBody.skus.salePrice",  # Lấy salePrice từ phần tử đầu tiên trong mảng skus
            "scraped_timestamp": "$scraped_timestamp",  # Thời gian scraped ngoài cấp item
            "scraped_date": {
                "$dateToString": {"format": "%Y-%m-%d", "date": {"$toDate": "$scraped_timestamp"}}
            }
        }
    },
    {
        "$group": {
            "_id": {"item_id": "$_id", "scraped_date": "$scraped_date"},  # Nhóm theo itemId và ngày scraped
            "prices": {"$push": "$salePrice"},  # Lấy danh sách giá từ trường salePrice
            "distinct_prices": {"$addToSet": "$salePrice"}  # Lấy các giá distinct
        }
    },
    {
        "$project": {
            "_id": 0,
            "item_id": "$_id.item_id",  # item_id
            "scraped_date": "$_id.scraped_date",  # scraped_date
            "prices": 1,  # Giá sản phẩm
            "distinct_prices_count": {"$size": "$distinct_prices"}  # Số lượng giá distinct trong ngày
        }
    },
    {
        "$sort": {
            "item_id": 1,  # Sắp xếp theo item_id
            "scraped_date": 1  # Sắp xếp theo ngày scraped
        }
    }
]

# Thực thi pipeline
result = list(lazada.aggregate(pipeline))  # Giả sử mongo_collection là tên collection của bạn

# Chuyển kết quả thành DataFrame
import pandas as pd
df = pd.DataFrame(result)

# Lấy danh sách các item_id duy nhất
unique_item_ids = df['item_id'].unique()

# Xóa namespace pandas sau khi sử dụng
del pd

In [88]:
df

,prices,item_id,scraped_date,distinct_prices_count
0,"[28990000, 18690000]",1040858590,2024-11-18,2
1,"[28990000, 18690000]",1040858590,2024-11-19,2
2,"[28990000, 18690000]",1040858590,2024-11-20,2
3,"[28990000, 18690000]",1040858590,2024-11-21,2
4,"[28990000, 18690000]",1040858590,2024-11-22,2
5,"[28990000, 18690000]",1040858590,2024-11-23,2
6,"[28990000, 18690000]",1040858590,2024-11-24,2
7,"[28990000, 18689000]",1040858590,2024-11-25,2
8,"[28990000, 18689000]",1040858590,2024-11-26,2
9,"[28990000, 18689000]",1040858590,2024-11-27,2


In [5]:
import glob
glob.glob('**/*2024-12-01*.json', recursive=True)

['data\\lazada\\1040858590_2024-12-01.json',
 'data\\lazada\\2730432618_2024-12-01.json',
 'data\\lazada\\2792189799_2024-12-01.json',
 'data\\shopee\\24509035804_2024-12-01.json',
 'data\\shopee\\27253416536_2024-12-01.json',
 'data\\shopee\\27312277956_2024-12-01.json',
 'data\\shopee\\27603632331_2024-12-01.json',
 'data\\shopee\\29560903606_2024-12-01.json',
 'data\\shopee\\5873954476_2024-12-01.json',
 'data\\tiki\\197665885_2024-12-01.json',
 'data\\tiki\\248280562_2024-12-01.json',
 'data\\tiki\\26140236_2024-12-01.json',
 'data\\tiki\\271362153_2024-12-01.json',
 'data\\tiki\\271973414_2024-12-01.json',
 'data\\tiki\\273258825_2024-12-01.json',
 'data\\tiki\\273880674_2024-12-01.json',
 'data\\tiki\\274037360_2024-12-01.json',
 'data\\tiki\\274057334_2024-12-01.json',
 'data\\tiki\\275078722_2024-12-01.json',
 'data\\tiki\\275127044_2024-12-01.json',
 'data\\tiki\\275716541_2024-12-01.json',
 'data\\tiki\\276425741_2024-12-01.json',
 'data\\tiki\\57422387_2024-12-01.json']